# NAS (Neural Architecture Search) for Classification on Vertex AI with TF-vision

Make sure that you have read the [required documentations](https://cloud.google.com/vertex-ai/docs/training/neural-architecture-search/overview#reading_order)
before executing this notebook.
NOTE: This notebook is meant to run pre-built trainer code with pre-built search-spaces. If you want to run your own trainer
code or create your own NAS search-space from scratch, then do not use this notebook.

This notebook shows example of [MnasNet](https://arxiv.org/abs/1807.11626) paper result on Imagenet data.
According to the paper, MnasNet achieves 75.2% top-1 accuracy with 78ms latency on a Pixel phone, 
which is 1.8x faster than MobileNetV2 with 0.5% higher accuracy and 2.3x faster than NASNet with 1.2% higher accuracy.
However, this notebook uses GPUs instead of TPUs for training and uses A100 80GB GPUs (a2-ultragpu-2) to evaluate latency.
With this notebook, the expected Stage2 top-1 accuracy on MNasnet is 75.2% with 50ms latency on A100 80GB GPUs (a2-ultragpu-2)).
The detailed settings for this notebook are:
- Stage-1 search
    - Number of trials: 10000
    - Number of GPUs per trial: 2
    - GPU type: NVIDIA_A100_80GB
    - Avg single trial training time: 1.5 hours
    - Number of parallel trials: 50
    - GPU quota used: 50*2 = 100 A100 80GB GPUs
    - Time to run: 7 days

- Stage-2 full-training with top 10 models
    - Number of trials: 10
    - Number of GPUs per trial: 2
    - GPU type: NVIDIA_A100_80GB
    - Avg single trial training time: 7 days
    - Number of parallel trials: 10
    - GPU quota used: 10*2 = 20 A100 80GB GPUs


You can also test drive MnasNet with just few trials with much lower cost.
See [here for the test drive settings and cost](https://cloud.google.com/vertex-ai/docs/training/neural-architecture-search/overview#mnasnet_test_drive).
For this use case, run this notebook only till section 'Sanity Check: Launch NAS stage 1 job with latency constraint'.


Here are the **pre-requisites** before you can start using this notebook: 
1. Your GCP project should have been (a) allow-listed and (b) [DWS GPU quota should have been allocated](https://cloud.google.com/vertex-ai/docs/training/neural-architecture-search/environment-setup#device-quota) for the NAS jobs.
2. You have selected a python3 kernel to run this notebook.

# Install required libraries

In [ ]:
%%sh

pip install tensorflow==2.7.0 --user
pip install tf-models-official==2.7.1
pip install pyglove==0.1.0

**NOTE: Please restart the notebook after installing above libraries successfully.**

# Download source code

This needs to be done just once.


In [ ]:
%%sh

# NOTE: It is ok for this step to fail if the directory exists.
mkdir -p ~/nas_experiment

In [ ]:
%%sh

rm -r -f ~/nas_experiment/nas_codes
git clone https://github.com/google/vertex-ai-nas.git ~/nas_experiment/nas_codes

# Set code path

In [ ]:
import os
os.chdir(os.path.join('/home/jupyter/nas_experiment/nas_codes'))

# Set up environment variables

Here we set up the environment variables.

NOTE: These have to be set-up every time you run a new session because the later code-blocks use them.


In [ ]:
# Set a unique USER name. This is used for creating a unique job-name identifier.
%env USER=<user>-nas
# Set a region to launch jobs into.
# If you only want to test-drive and do not have enough GPU quota, then you can use 'us-central1' region
# which should have a default quota of 12 Nvidia T4 GPUs.
%env REGION=us-central1
# Set any unique docker-id for this run. When the next section builds a docker, then this id will be used to tag it.
%env TRAINER_DOCKER_ID=<user>-nas-trainer-dockerid
%env LATENCY_CALCULATOR_DOCKER_ID=<user>-nas-calc-dockerid
# The GCP project-id must be the one that has been clear-listed for the NAS jobs. 
%env PROJECT_ID=<project_id>
# Set an output working directory for the NAS jobs. The GCP project should have write access to 
# this GCS output directory. A simple way to ensure this is to use a bucket inside the same GCP project.
# NOTE: The region of the bucket must be the same as job's.
%env GCS_ROOT_DIR=<GCS directory to place results>
# Set the accelerator device type.
%env DEVICE=NVIDIA_A100_80GB
# Set the GCS paths to the training and validation datasets. The GCP project should have read access to the data-location.
# Please read the "Data-Preparation" section 
# (https://cloud.google.com/vertex-ai/docs/training/neural-architecture-search/pre-built-trainer#data-preparation)
# in the documentation to ensure that the data is in an appropriate format
# suitable for the NAS pipeline. The documentation also mentions how you can download and prepare the ImageNet dataset.
# You can run the "Validate and Visualize data format" section in this notebook 
# to verify that the data can be loaded properly.
# Update the path to ImageNet data below.
%env STAGE1_TRAINING_DATA_PATH=<imagenet-data-gs-path>/classification/imagenet/train-00[0-8]??-of-01024
%env STAGE1_VALIDATION_DATA_PATH=<imagenet-data-gs-path>/classification/imagenet/train-009??-of-01024,gs://cloud-nas-public-eu/classification/imagenet/train-01???-of-01024
%env STAGE2_TRAINING_DATA_PATH=<imagenet-data-gs-path>/classification/imagenet/train*
%env STAGE2_VALIDATION_DATA_PATH=<imagenet-data-gs-path>/classification/imagenet/validation*

**NOTE:** The following set up steps need to be done just once.

In [ ]:
%%sh

# Authenticate docker for your artifact registry.
gcloud auth configure-docker ${REGION}-docker.pkg.dev

In [ ]:
%%sh

# NOTE: This needs to be just once for the first time. It is ok for this to FAIL if the GCS bucket already exists.

# Create the output directory. 
gsutil mkdir $GCS_ROOT_DIR

# Build container
The container must be built the first time and then every time the source-code is modified. Otherwise, there is no need to run this step. This step internally builds the *Dockerfile* in the source-code directory and then pushes the docker to the cloud. 

In [ ]:
%%sh

# NOTE: This step can take several minutes when run for the first time.

python3 vertex_nas_cli.py build \
--project_id=${PROJECT_ID} \
--trainer_docker_id=${TRAINER_DOCKER_ID} \
--region=${REGION} \
--trainer_docker_file="tf_vision/nas_multi_trial.Dockerfile" \
--latency_calculator_docker_id=${LATENCY_CALCULATOR_DOCKER_ID} \
--latency_calculator_docker_file="tf_vision/latency_computation_using_saved_model.Dockerfile"

# Sanity Check: Launch NAS stage 1 job with latency constraint
If you do not want to run a full MNasNet run and only want to test drive with [just few trials as described here,](https://cloud.google.com/vertex-ai/docs/training/neural-architecture-search/overview#mnasnet_test_drive)
then only run the following command and skip the rest of the notebook.

In [ ]:
%%sh

DATE="$(date '+%Y%m%d_%H%M%S')"
JOB_ID="${USER}_nas_tfvision_icn_latency_${DATE}"


# GPU configs
# https://cloud.google.com/vertex-ai/docs/training/configure-compute#specifying_gpus


CMD="
python3 vertex_nas_cli.py search \
--project_id=${PROJECT_ID} \
--region=${REGION} \
--trainer_docker_id=${TRAINER_DOCKER_ID} \
--job_name=${JOB_ID} \
--max_nas_trial=10 \
--max_parallel_nas_trial=10 \
--max_failed_nas_trial=10 \
--use_prebuilt_trainer=True \
--prebuilt_search_space="mnasnet" \
--master_machine_type="a2-ultragpu-2g" \
--accelerator_type=${DEVICE} \
--num_gpus=2 \
--root_output_dir=${GCS_ROOT_DIR} \
--latency_calculator_docker_id=${LATENCY_CALCULATOR_DOCKER_ID} \
--target_device_type=CPU \
--use_prebuilt_latency_calculator=True \
--search_docker_flags \
params_override="tf_vision/configs/experiments/mnasnet_search_gpu.yaml" \
training_data_path=${STAGE1_TRAINING_DATA_PATH} \
validation_data_path=${STAGE1_VALIDATION_DATA_PATH} \
model="classification" \
target_device_latency_ms=50
"

echo Executing command: ${CMD}
    
${CMD}

# Full Experiment: Launch NAS stage 1 job with latency constraint
If you do not want to run a full MNasNet run and only want to test drive with [just few trials as described here,](https://cloud.google.com/vertex-ai/docs/training/neural-architecture-search/overview#mnasnet_test_drive)
then only run the following command and skip the rest of the notebook.

In [ ]:
%%sh

DATE="$(date '+%Y%m%d_%H%M%S')"
JOB_ID="${USER}_nas_tfvision_icn_latency_${DATE}"


# GPU configs
# https://cloud.google.com/vertex-ai/docs/training/configure-compute#specifying_gpus


CMD="
python3 vertex_nas_cli.py search \
--project_id=${PROJECT_ID} \
--region=${REGION} \
--trainer_docker_id=${TRAINER_DOCKER_ID} \
--job_name=${JOB_ID} \
--max_nas_trial=10000 \
--max_parallel_nas_trial=30 \
--max_failed_nas_trial=3000 \
--use_prebuilt_trainer=True \
--prebuilt_search_space="mnasnet" \
--master_machine_type="a2-ultragpu-2g" \
--accelerator_type=${DEVICE} \
--num_gpus=2 \
--root_output_dir=${GCS_ROOT_DIR} \
--latency_calculator_docker_id=${LATENCY_CALCULATOR_DOCKER_ID} \
--target_device_type=CPU \
--use_prebuilt_latency_calculator=True \
--search_docker_flags \
params_override="tf_vision/configs/experiments/mnasnet_search_gpu.yaml" \
training_data_path=${STAGE1_TRAINING_DATA_PATH} \
validation_data_path=${STAGE1_VALIDATION_DATA_PATH} \
model="classification" \
target_device_latency_ms=50
"

echo Executing command: ${CMD}
    
${CMD}

# Resume NAS stage 1 job with latency constraint
You can find the nas_job_id and latency_job_id numbers from the logs

In [ ]:
%%sh

previous_nas_job_id=<fill>
previous_latency_job_id=<fill>
DATE="$(date '+%Y%m%d_%H%M%S')"
JOB_NAME="${USER}_nas_tfvision_icn_latency_${DATE}"


CMD="python3 vertex_nas_cli.py search_resume \
  --project_id=${PROJECT_ID} \
  --region=${REGION} \
  --job_name=${JOB_NAME} \
  --previous_nas_job_id=${previous_nas_job_id} \
  --previous_latency_job_id=${previous_latency_job_id} \
  --root_output_dir=${GCS_ROOT_DIR} \
  --max_nas_trial=2 \
  --max_parallel_nas_trial=2 \
  --max_failed_nas_trial=2"
echo Executing command: ${CMD}
    
${CMD}

# Inspect NAS search progress
A periodic evaluation while the search is going on can help decide if the search job has converged. This code-block shows how to generate summary of top N trials so far.

In [ ]:
# Set the stage1 search-job id. It's a numeric value returned by the Vertex service.
%env JOB_ID=<fill>

In [ ]:
%%sh

mkdir -p /home/jupyter/nas_experiment/jobs
python3 vertex_nas_cli.py list_trials \
--project_id=${PROJECT_ID} \
--job_id=${JOB_ID} \
--region=${REGION} \
--trials_output_file=/home/jupyter/nas_experiment/jobs/${JOB_ID}.yaml

cat /home/jupyter/nas_experiment/jobs/${JOB_ID}.yaml

# Launch NAS stage 2 job

In [ ]:
%%sh

DATE="$(date '+%Y%m%d_%H%M%S')"

# Please modify the "JOB_ID", "TRIAL_ID", and the finetuning config file before running.
# JOB_ID is numeric value you can find from the job UI in Pantheon.
JOB_ID=<fill>
# TRIAL_ID is one of the best performing trials which has to be finetuned.
TRIAL_IDS=<fill> # The top trials chosen for training to converge.

CMD="

python3 vertex_nas_cli.py train \
--project_id=${PROJECT_ID} \
--region=${REGION} \
--trainer_docker_id=${TRAINER_DOCKER_ID} \
--use_prebuilt_trainer=True \
--prebuilt_search_space="mnasnet" \
--train_accelerator_type=${DEVICE} \
--train_num_gpus=4 \
--root_output_dir=${GCS_ROOT_DIR} \
--search_job_id=${JOB_ID} \
--search_job_region=${REGION} \
--train_nas_trial_numbers=${TRIAL_IDS} \
--train_job_suffix="stage2_${DATE}" \
--train_docker_flags \
params_override="tf_vision/configs/experiments/mnasnet_search_finetune_gpu.yaml" \
training_data_path=${STAGE2_TRAINING_DATA_PATH} \
validation_data_path=${STAGE2_VALIDATION_DATA_PATH} \
model="classification"
"

echo Executing command: ${CMD}
    
${CMD}